# 2 Train GAN

## 2.1 Import the libraries used in the model

#### Notes: 
* Need to use TensoFlow 1.5.0 with keras 2.1.2. Otherwise, there is a problem with LeakyReLU
* Syntax: pip install tensorflow==1.5.0  and pip install keras==2.1.2

In [13]:
from __future__ import print_function

from collections import defaultdict
try:
    import cPickle as pickle
except ImportError:
    import pickle
from PIL import Image

from six.moves import range

import tensorflow as tf
import keras.backend as K
from keras.datasets import mnist
from keras import layers
from keras.layers import Input, Dense, Reshape, Flatten, Embedding, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils.generic_utils import Progbar
import numpy as np
import random as rn
import os
import argparse
import time

import sys
sys.path.insert(0,'./privacy_accountant')
import utils, accountant

## 2.2 Set model parameters

In [14]:
from collections import defaultdict

directory = ('./output/')
if not os.path.exists(directory):
    os.mkdir(directory)

epochs = 5  # Should be 500
batch_size = 1
latent_size = 100

# setting seed for reproducibility
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)
rn.seed(seed)

# Adam parameters suggested in https://arxiv.org/abs/1511.06434
adam_lr = 2e-05
adam_beta_1 = 0.5

training_size = 600
K.set_image_data_format('channels_first')

## 2.3 Build the discriminator

In [15]:
def build_discriminator():
    # build a relatively standard conv net, with LeakyReLUs as suggested in
    # the reference paper
    print('Discriminator')
    cnn = Sequential()
    cnn.add(Conv2D(32, 3, padding='same', strides=2,
                   input_shape=(1, 3, 12)))
    cnn.add(LeakyReLU())
    cnn.add(Dropout(0.3))

    cnn.add(Conv2D(64, 3, padding='same', strides=1))
    cnn.add(LeakyReLU())
    cnn.add(Dropout(0.3))

    cnn.add(Flatten())
    cnn.add(Dense(1024, activation='relu'))
    cnn.add(Dropout(0.3))
    cnn.add(Dense(1024, activation='relu'))
    patient = Input(shape=(1, 3, 12))

    features = cnn(patient)
    cnn.summary()

    fake = Dense(1, activation='sigmoid', name='generation')(features)
    # aux could probably be 1 sigmoid too...
    aux = Dense(2, activation='softmax', name='auxiliary')(features)

    return Model(patient, [fake, aux])

## 2.4 Build the generator

In [16]:
def build_generator(latent_size):
    # we will map a pair of (z, L), where z is a latent vector and L is a
    # label drawn from P_c, to image space (..., 1, 28, 28)
    print('Generator')
    cnn = Sequential()

    cnn.add(Dense(256, input_dim=latent_size, activation='relu'))
    cnn.add(Dense(32 * 3 * 4, activation='relu'))
    cnn.add(Reshape((32, 3, 4)))

    # upsample to (..., 14, 14)
    cnn.add(UpSampling2D(size=(2, 6)))
    cnn.add(Conv2D(256, 5, padding='same',
                   activation='relu',
                   kernel_initializer='glorot_normal'))

    # take a channel axis reduction
    cnn.add(Conv2D(1, 4, strides=2, padding='same',
                   activation='linear',
                   kernel_initializer='glorot_normal'))

    # dense layer to reshape
    cnn.summary()

    # this is the z space commonly refered to in GAN papers
    latent = Input(shape=(latent_size, ))

    # this will be our label
    patient_class = Input(shape=(1,), dtype='int32')

    # 10 classes in MNIST
    cls = Flatten()(Embedding(
                        2, latent_size,
                        embeddings_initializer='glorot_normal')(patient_class))

    # hadamard product between z-space and a class conditional embedding
    h = layers.multiply([latent, cls])

    fake_patient = cnn(h)

    return Model([latent, patient_class], fake_patient)

## 2.5 Train the model

In [18]:
# Train the discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
                                     loss=['binary_crossentropy','sparse_categorical_crossentropy'])

# Train the generator
generator = build_generator(latent_size)
generator.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
                                 loss='binary_crossentropy')

latent = Input(shape=(latent_size, ))
image_class = Input(shape=(1,), dtype='int32')

# get a fake image
fake = generator([latent, image_class])

# we only want to be able to train generation for the combined model
discriminator.trainable = False
fake, aux = discriminator(fake)
combined = Model([latent, image_class], [fake, aux])

combined.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'])

Discriminator
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 2, 6)          320       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 2, 6)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 32, 2, 6)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 2, 6)          18496     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 64, 2, 6)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 64, 2, 6)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 768)               0      

## 2.6 Feed the input data into the model

In [19]:
import pickle as pkl

X_input = pickle.load(open('./output/X_processed.pkl', 'rb'))
y_input = pickle.load(open('./output/y_processed.pkl', 'rb'))
print(X_input.shape, y_input.shape)

X_train = X_input[:training_size]
X_test = X_input[training_size:]
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

y_train = y_input[:training_size]
y_test = y_input[training_size:]

num_train, num_test = X_train.shape[0], X_test.shape[0]

train_history = defaultdict(list)
test_history = defaultdict(list)
privacy_history = []

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

(839, 3, 12) (839,)


In [ ]:
init_op = tf.initialize_all_variables()

with tf.Session(config=config) as sess:
    sess.run(init_op)
    eps = tf.placeholder(tf.float32)
    delta = tf.placeholder(tf.float32)

    for epoch in range(epochs):
        print('Epoch {} of {}'.format(epoch + 1, epochs))

        num_batches = training_size
        progress_bar = Progbar(target=num_batches)

        random_sample = np.random.randint(0, training_size,
                                              size=training_size)

        epoch_gen_loss = []
        epoch_disc_loss = []

        train_start_time = time.clock()
        for index in range(num_batches):
            progress_bar.update(index)
            # generate a new batch of noise
            noise = np.random.uniform(-1, 1, (batch_size, latent_size))

            # get a batch of real patients
            image_batch = np.expand_dims(X_train[random_sample[index]], axis=1)
            label_batch = np.expand_dims(y_train[random_sample[index]], axis=1)

            # sample some labels from p_c
            sampled_labels = np.random.randint(0, 2, batch_size)

            # generate a batch of fake patients, using the generated labels as a
            # conditioner. We reshape the sampled labels to be
            # (batch_size, 1) so that we can feed them into the embedding
            # layer as a length one sequence
            generated_images = generator.predict([noise, sampled_labels.reshape((-1, 1))], verbose=0)

            X = np.concatenate((image_batch, generated_images))
            y = np.array([1] * batch_size + [0] * batch_size)
            aux_y = np.concatenate((label_batch, sampled_labels), axis=0)

            epoch_disc_loss.append(discriminator.train_on_batch(X, [y, aux_y]))

            # make new noise. we generate 2 * batch size here such that we have
            # the generator optimize over an identical number of patients as the
            # discriminator
            noise = np.random.uniform(-1, 1, (2 * batch_size, latent_size))
            sampled_labels = np.random.randint(0, 2, 2 * batch_size)

            # we want to train the generator to trick the discriminator
            # For the generator, we want all the {fake, not-fake} labels to say
            # not-fake
            trick = np.ones(2 * batch_size)

            epoch_gen_loss.append(combined.train_on_batch(
                    [noise, sampled_labels.reshape((-1, 1))],
                    [trick, sampled_labels]))
            print('\n Train time: ', time.clock() - train_start_time)
            print('accum privacy, batches: ' + str(num_batches))
            priv_start_time = time.clock()

            # separate privacy accumulation for speed
            #privacy_accum_op = priv_accountant.accumulate_privacy_spending(
            #    [None, None], args.noise, batch_size)
            #for index in range(num_batches):
            #    with tf.control_dependencies([privacy_accum_op]):
            #        spent_eps_deltas = priv_accountant.get_privacy_spent(
            #            sess, target_eps=target_eps)
            #        privacy_history.append(spent_eps_deltas)
            #    sess.run([privacy_accum_op])

            #for spent_eps, spent_delta in spent_eps_deltas:
            #    print("spent privacy: eps %.4f delta %.5g" % (
            #        spent_eps, spent_delta))
            #print('priv time: ', time.clock() - priv_start_time)

            #if spent_eps_deltas[-3][1] > 0.0001:
            #    raise Exception('spent privacy')

            print('\nTesting for epoch {}:'.format(epoch + 1))
            # generate a new batch of noise
            noise = np.random.uniform(-1, 1, (num_test, latent_size))

            # sample some labels from p_c and generate patients from them
            sampled_labels = np.random.randint(0, 2, num_test)
            generated_images = generator.predict(
                [noise, sampled_labels.reshape((-1, 1))], verbose=False)

            print(sampled_labels[0])
            print(generated_images[0].astype(int))

            X = np.concatenate((X_test, generated_images))
            y = np.array([1] * num_test + [0] * num_test)
            aux_y = np.concatenate((y_test, sampled_labels), axis=0)

            # see if the discriminator can figure itself out...
            discriminator_test_loss = discriminator.evaluate(
                X, [y, aux_y], verbose=False)

            discriminator_train_loss = np.mean(np.array(epoch_disc_loss), axis=0)

            # make new noise
            noise = np.random.uniform(-1, 1, (2 * num_test, latent_size))
            sampled_labels = np.random.randint(0, 2, 2 * num_test)

            trick = np.ones(2 * num_test)

            generator_test_loss = combined.evaluate(
                [noise, sampled_labels.reshape((-1, 1))],
                [trick, sampled_labels], verbose=False)

            generator_train_loss = np.mean(np.array(epoch_gen_loss), axis=0)

            # generate an epoch report on performance
            train_history['generator'].append(generator_train_loss)
            train_history['discriminator'].append(discriminator_train_loss)

            test_history['generator'].append(generator_test_loss)
            test_history['discriminator'].append(discriminator_test_loss)

            print('{0:<22s} | {1:4s} | {2:15s} | {3:5s}'.format(
                'component', *discriminator.metrics_names))
            print('-' * 65)

            ROW_FMT = '{0:<22s} | {1:<4.2f} | {2:<15.2f} | {3:<5.2f}'
            print(ROW_FMT.format('generator (train)',
                                 *train_history['generator'][-1]))
            print(ROW_FMT.format('generator (test)',
                                 *test_history['generator'][-1]))
            print(ROW_FMT.format('discriminator (train)',
                                 *train_history['discriminator'][-1]))
            print(ROW_FMT.format('discriminator (test)',
                                 *test_history['discriminator'][-1]))
            generator.save(
                directory +
                'params_generator_epoch_{0:03d}.h5'.format(epoch))

            if epoch > (epochs-10):
                discriminator.save(
                    directory +
                    'params_discriminator_epoch_{0:03d}.h5'.format(epoch))

            pickle.dump({'train': train_history, 'test': test_history,
                         'privacy': privacy_history},open(directory + 'acgan-history.pkl', 'wb'))
 

Epoch 1 of 5
  0/600 [..............................] - ETA: 0s

C:\Users\Lanthao\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
C:\Users\Lanthao\Anaconda3\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'



 Train time:  0.5670046320894926
accum privacy, batches: 600

Testing for epoch 1:
1
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.39 | 0.69            | 0.69 
generator (test)       | 1.39 | 0.69            | 0.69 
discriminator (train)  | 1.35 | 0.65            | 0.70 
discriminator (test)   | 1.39 | 0.66            | 0.73 
  1/600 [..............................] - ETA: 47:35
 Train time:  4.866122190211854
accum privacy, batches: 600

Testing for epoch 1:
1
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.39 | 0.69            | 0.69 
generator (test)       | 1.39 | 0.69            | 0.69 
discri

0
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.39 | 0.69            | 0.69 
generator (test)       | 1.39 | 0.69            | 0.69 
discriminator (train)  | 1.32 | 0.63            | 0.69 
discriminator (test)   | 1.30 | 0.64            | 0.66 
 15/600 [..............................] - ETA: 35:16
 Train time:  54.36631522683979
accum privacy, batches: 600

Testing for epoch 1:
0
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.39 | 0.69            | 0.69 
generator (test)       | 1.39 | 0.69            | 0.69 
discriminator (train)  | 1.33 | 0.64            | 0.69 
discriminator (test)   | 1.30 | 0.

component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.39 | 0.69            | 0.69 
generator (test)       | 1.39 | 0.69            | 0.69 
discriminator (train)  | 1.29 | 0.63            | 0.66 
discriminator (test)   | 1.23 | 0.62            | 0.61 
 29/600 [>.............................] - ETA: 33:48
 Train time:  103.09937399287378
accum privacy, batches: 600

Testing for epoch 1:
1
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.39 | 0.69            | 0.69 
generator (test)       | 1.39 | 0.69            | 0.69 
discriminator (train)  | 1.28 | 0.63            | 0.66 
discriminator (test)   | 1.23 | 0.62            | 0.61 
 30/600 [>.............................] - ETA: 33:43
 Train time

...

component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.78 | 1.12            | 0.66 
generator (test)       | 1.91 | 1.36            | 0.55 
discriminator (train)  | 0.81 | 0.39            | 0.41 
discriminator (test)   | 0.65 | 0.34            | 0.31 
315/600 [==============>...............] - ETA: 17:11
 Train time:  1139.666735486133
accum privacy, batches: 600

Testing for epoch 5:
0
[[[0 0 1 0 1 0 1 1 1 0 1 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 1 1 1 1 1 1 1 1]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.78 | 1.12            | 0.66 
generator (test)       | 1.91 | 1.38            | 0.54 
discriminator (train)  | 0.81 | 0.39            | 0.41 
discriminator (test)   | 0.65 | 0.34            | 0.31 
316/600 [==============>...............] - ETA: 17:07
 Train time:

329/600 [===============>..............] - ETA: 16:19
 Train time:  1189.3013274130171
accum privacy, batches: 600

Testing for epoch 5:
1
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.79 | 1.13            | 0.66 
generator (test)       | 2.01 | 1.46            | 0.55 
discriminator (train)  | 0.81 | 0.39            | 0.41 
discriminator (test)   | 0.65 | 0.33            | 0.32 
330/600 [===============>..............] - ETA: 16:15
 Train time:  1192.866029816856
accum privacy, batches: 600

Testing for epoch 5:
1
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.79 | 1.13            | 0.66 
generator

0
[[[1 1 1 0 1 0 1 1 1 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 1 1 1 1 1 1 1 1]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.79 | 1.13            | 0.65 
generator (test)       | 1.99 | 1.46            | 0.53 
discriminator (train)  | 0.81 | 0.39            | 0.41 
discriminator (test)   | 0.63 | 0.32            | 0.30 
344/600 [================>.............] - ETA: 15:24
 Train time:  1242.5124254006205
accum privacy, batches: 600

Testing for epoch 5:
1
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 1 1 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.79 | 1.13            | 0.65 
generator (test)       | 1.96 | 1.45            | 0.51 
discriminator (train)  | 0.80 | 0.39            | 0.41 
discriminator (test)   | 0.63 | 0

component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.80 | 1.14            | 0.66 
generator (test)       | 2.03 | 1.50            | 0.54 
discriminator (train)  | 0.80 | 0.39            | 0.41 
discriminator (test)   | 0.61 | 0.31            | 0.31 
358/600 [================>.............] - ETA: 14:33
 Train time:  1292.1992418279187
accum privacy, batches: 600

Testing for epoch 5:
0
[[[1 1 1 1 1 1 1 1 1 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 1 1 1 1 1 1 1 1]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.80 | 1.14            | 0.66 
generator (test)       | 2.05 | 1.52            | 0.53 
discriminator (train)  | 0.80 | 0.39            | 0.41 
discriminator (test)   | 0.62 | 0.30            | 0.31 
359/600 [================>.............] - ETA: 14:29
 Train time

372/600 [=================>............] - ETA: 13:43
 Train time:  1343.0143202386862
accum privacy, batches: 600

Testing for epoch 5:
1
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.80 | 1.15            | 0.65 
generator (test)       | 2.21 | 1.67            | 0.54 
discriminator (train)  | 0.80 | 0.39            | 0.41 
discriminator (test)   | 0.63 | 0.31            | 0.32 
373/600 [=================>............] - ETA: 13:39
 Train time:  1346.6748071015372
accum privacy, batches: 600

Testing for epoch 5:
0
[[[1 1 1 1 1 1 1 1 1 0 1 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 1 1 1 1 1 1 1 1 1 1]]]
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.80 | 1.15            | 0.65 
generato